In [48]:
import numpy as np
import numba
import itertools
import sys

# Speed/Memory Tradeoff for using Python ints vs. Numpy.uint8

In [56]:
%timeit 5

6.36 ns ± 0.086 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [57]:
sys.getsizeof(5)

28

In [58]:
%timeit np.uint8(5)

260 ns ± 5.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [59]:
sys.getsizeof(np.uint8(5))

25

All those bytes are going to add up, but no one will ever notice the time.

In [60]:
%timeit 2 + 2

9.33 ns ± 0.231 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [61]:
%timeit 2 + np.uint(2)

3.28 µs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [78]:
%timeit np.uint8(2) + np.uint8(2)

565 ns ± 11.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Pretty sure addition will never be done but clearly arithmetic operations are better with with `np.uint8`'s

# Initial card array building

In [69]:
%timeit np.array([0,0,0,0,0], dtype=np.uint8)

1.78 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [71]:
%timeit np.array([0,0,0,0,0], dtype=int)

1.66 µs ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
%timeit np.zeros(5, dtype=np.uint8)

1.08 µs ± 5.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [45]:
%timeit np.zeros(np.uint(5), dtype=np.uint8)

1.6 µs ± 6.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [63]:
%timeit np.zeros(np.uint(5), dtype=int)

1.51 µs ± 9.59 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [64]:
%timeit np.zeros(5, dtype=int)

999 ns ± 7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


There is no point in using `np.uint8`'s as arguments.

In [76]:
sys.getsizeof(np.zeros(5, dtype=np.uint8))

101

In [77]:
sys.getsizeof(np.zeros(5, dtype=int))

136

I'd say the size difference of the different int types is worth the ~600 nanosecond difference in building time.

# Hashing

In [137]:
np.random.seed(12341234)
arr = np.random.randint(low=1, high=53, size=5)

In [138]:
%timeit hash(str(arr))

43.9 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [139]:
%timeit hash(arr.tostring())

170 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [140]:
%timeit hash(str(arr.data))

926 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [141]:
%timeit hash(str(arr.data.tobytes))

1.21 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


`.tostring()` is the winner.

## Prove completely uncolliding hashes

In [43]:
combo_dict = {}

In [44]:
cards = [0] * 5 + list(range(1, 53))

In [45]:
possible_hands_with_repeats = itertools.combinations(cards, 5)

In [46]:
possible_hands = set(possible_hands_with_repeats)

In [47]:
for hand in possible_hands:
    curr_hand = np.array(hand, dtype=np.uint8)
    curr_hash = hash(curr_hand.tostring())
    if curr_hash in combo_dict:
        print(f"already in: {curr_hash}, {combo_dict[curr_hash]}")
        print(f"to insert: {curr_hash}, {curr_hand}")
        break
    else:
        combo_dict[curr_hash] = curr_hand

# Misc

In [73]:
np.array(list(itertools.combinations([1, 2, 3, 4], 2)))

array([[1, 2],
       [1, 3],
       [1, 4],
       [2, 3],
       [2, 4],
       [3, 4]])

In [84]:
a = []
a.extend(itertools.combinations([1, 2, 3, 4], 2))
a.extend(itertools.combinations([5, 6, 7, 8], 2))
a

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (5, 6),
 (5, 7),
 (5, 8),
 (6, 7),
 (6, 8),
 (7, 8)]

In [89]:
cards_but_not_hand = []
suits = np.arange(1, 53, dtype=np.uint8).reshape(13, 4)
for suit in suits:
    cards_but_not_hand.extend(itertools.combinations(suit, 2))

In [94]:
ye = []
for suit in suits:
    ye.extend(itertools.combinations(suit, 1))
ye

[(1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (6,),
 (7,),
 (8,),
 (9,),
 (10,),
 (11,),
 (12,),
 (13,),
 (14,),
 (15,),
 (16,),
 (17,),
 (18,),
 (19,),
 (20,),
 (21,),
 (22,),
 (23,),
 (24,),
 (25,),
 (26,),
 (27,),
 (28,),
 (29,),
 (30,),
 (31,),
 (32,),
 (33,),
 (34,),
 (35,),
 (36,),
 (37,),
 (38,),
 (39,),
 (40,),
 (41,),
 (42,),
 (43,),
 (44,),
 (45,),
 (46,),
 (47,),
 (48,),
 (49,),
 (50,),
 (51,),
 (52,)]

In [91]:
np.array(cards_but_not_hand, dtype=np.uint8)

array([[ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 2,  3],
       [ 2,  4],
       [ 3,  4],
       [ 5,  6],
       [ 5,  7],
       [ 5,  8],
       [ 6,  7],
       [ 6,  8],
       [ 7,  8],
       [ 9, 10],
       [ 9, 11],
       [ 9, 12],
       [10, 11],
       [10, 12],
       [11, 12],
       [13, 14],
       [13, 15],
       [13, 16],
       [14, 15],
       [14, 16],
       [15, 16],
       [17, 18],
       [17, 19],
       [17, 20],
       [18, 19],
       [18, 20],
       [19, 20],
       [21, 22],
       [21, 23],
       [21, 24],
       [22, 23],
       [22, 24],
       [23, 24],
       [25, 26],
       [25, 27],
       [25, 28],
       [26, 27],
       [26, 28],
       [27, 28],
       [29, 30],
       [29, 31],
       [29, 32],
       [30, 31],
       [30, 32],
       [31, 32],
       [33, 34],
       [33, 35],
       [33, 36],
       [34, 35],
       [34, 36],
       [35, 36],
       [37, 38],
       [37, 39],
       [37, 40],
       [38, 39],
       [38, 40

In [88]:
np.arange(1, 53, dtype=np.uint8).reshape(13, 4)

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20],
       [21, 22, 23, 24],
       [25, 26, 27, 28],
       [29, 30, 31, 32],
       [33, 34, 35, 36],
       [37, 38, 39, 40],
       [41, 42, 43, 44],
       [45, 46, 47, 48],
       [49, 50, 51, 52]], dtype=uint8)

In [69]:
a

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [71]:
a[:, 3:5] = b

In [72]:
a

array([[0, 0, 0, 1, 2],
       [0, 0, 0, 1, 3],
       [0, 0, 0, 1, 4],
       [0, 0, 0, 2, 3],
       [0, 0, 0, 2, 4],
       [0, 0, 0, 3, 4]], dtype=uint8)